<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#housekeeping" data-toc-modified-id="housekeeping-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>housekeeping</a></span><ul class="toc-item"><li><span><a href="#languages-&amp;-packages" data-toc-modified-id="languages-&amp;-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>languages &amp; packages</a></span></li><li><span><a href="#env-set-up" data-toc-modified-id="env-set-up-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>env set-up</a></span></li><li><span><a href="#model-function" data-toc-modified-id="model-function-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>model function</a></span></li></ul></li><li><span><a href="#step-by-step-numerical-solve-via-SymPy" data-toc-modified-id="step-by-step-numerical-solve-via-SymPy-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>step-by-step numerical solve via SymPy</a></span><ul class="toc-item"><li><span><a href="#environmental-preparation" data-toc-modified-id="environmental-preparation-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>environmental preparation</a></span></li><li><span><a href="#solve-each-parameter-in-each-equation" data-toc-modified-id="solve-each-parameter-in-each-equation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>solve each parameter in each equation</a></span><ul class="toc-item"><li><span><a href="#$g_p$:-in-$dp$-&amp;-$dm$" data-toc-modified-id="$g_p$:-in-$dp$-&amp;-$dm$-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>$g_p$: in $dp$ &amp; $dm$</a></span></li><li><span><a href="#$e_p$:-in-$dp$-&amp;-$dm$" data-toc-modified-id="$e_p$:-in-$dp$-&amp;-$dm$-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>$e_p$: in $dp$ &amp; $dm$</a></span></li><li><span><a href="#$R_p$:-in-$dp$" data-toc-modified-id="$R_p$:-in-$dp$-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>$R_p$: in $dp$</a></span></li><li><span><a href="#$m_p$:-in-dp-&amp;-dm" data-toc-modified-id="$m_p$:-in-dp-&amp;-dm-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>$m_p$: in dp &amp; dm</a></span></li><li><span><a href="#$g_c$:-in-$dc$-&amp;-$dm$" data-toc-modified-id="$g_c$:-in-$dc$-&amp;-$dm$-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>$g_c$: in $dc$ &amp; $dm$</a></span></li><li><span><a href="#$e_c$:-in-$dc$-&amp;-$dm$" data-toc-modified-id="$e_c$:-in-$dc$-&amp;-$dm$-2.2.6"><span class="toc-item-num">2.2.6&nbsp;&nbsp;</span>$e_c$: in $dc$ &amp; $dm$</a></span></li><li><span><a href="#$R_c$:-in-$dc$" data-toc-modified-id="$R_c$:-in-$dc$-2.2.7"><span class="toc-item-num">2.2.7&nbsp;&nbsp;</span>$R_c$: in $dc$</a></span></li><li><span><a href="#$m_c$:-in-$dc$-&amp;-$dm$" data-toc-modified-id="$m_c$:-in-$dc$-&amp;-$dm$-2.2.8"><span class="toc-item-num">2.2.8&nbsp;&nbsp;</span>$m_c$: in $dc$ &amp; $dm$</a></span></li></ul></li></ul></li></ul></div>

# housekeeping
This notebook is for analytically solving parameters in the set of ODEs for eco-bioelectric cell (EBC).

## languages & packages
Julia-lang 1.3.1: `PyCall` (1.91.2), `DataFrames` (0.20.0), `SymPy` (1.0.18)

## env set-up

In [1]:
using PyCall, DataFrames, SymPy
mt = pyimport("math")
sc = pyimport("scipy")
itg = pyimport("scipy.integrate")
sympy.init_printing() ## use other py3 function not implemented in native Julia-lang
println()

## model function
![](graph/PBD_00.png)

In [2]:
function ebc0(popDen, t, gp,ep,Rp,mp, gc,ec,Rc,mc)
    
    ## population densities
    P = popDen[:1]
    M = popDen[:2]
    C = popDen[:3]
    
    ## numerical adaptation to net growth
    #if gp <1; gp = 1+gp;end
    #if gc <1; gc = 1+gc;end
    
    ## fluctuation model
    dp = P*(gp*ep - Rp - mp)
    dm = P*(gp*(1-ep) + mp) + C*(gc*(1-ec)*M + mc)
    dc = C*(gc*ec*M - Rc - mc)
    
    ## non-recoverable elimination
    if P <=0; dp=0;end
    if C <=0; dc=0;end
    
    return(sc.array([dp,dm,dc]))
end

ebc0 (generic function with 1 method)

# step-by-step numerical solve via SymPy
This step-wise workflow is following CMEE [notes](https://nbviewer.jupyter.org/github/mhasoba/TheMulQuaBio/blob/master/notebooks/Appendix-Maths.ipynb) (last section) from Samraat on Jupyter math models
## environmental preparation
define variables

In [3]:
@vars P; @vars M; @vars C
@vars gp; @vars ep; @vars Rp; @vars mp
@vars gc; @vars ec; @vars Rc; @vars mc

(mc,)

define SymPy ODE system

In [16]:
dp = P*(gp*ep - Rp - mp)
dm = P*(gp*(1-ep) + mp) + C*(gc*(1-ec)*M + mc)
dc = C*(gc*ec*M - Rc - mc)
dp, dm, dc

(P*(-Rp + ep*gp - mp), C*(M*gc*(1 - ec) + mc) + P*(gp*(1 - ep) + mp), C*(M*ec*gc - Rc - mc))

In [15]:
dp

P⋅(-Rp + ep⋅gp - mp)

In [10]:
dm

C⋅(M⋅gc⋅(1 - ec) + mc) + P⋅(gp⋅(1 - ep) + mp)

In [11]:
dc

C⋅(M⋅ec⋅gc - Rc - mc)

define eqm state

In [5]:
p_eqm = Eq(dp, 0)
m_eqm = Eq(dm, 0)
c_eqm = Eq(dc, 0)
p_eqm, m_eqm, c_eqm

(Eq(P*(-Rp + ep*gp - mp), 0), Eq(C*(M*gc*(1 - ec) + mc) + P*(gp*(1 - ep) + mp), 0), Eq(C*(M*ec*gc - Rc - mc), 0))

In [12]:
p_eqm

P⋅(-Rp + ep⋅gp - mp) = 0

In [13]:
m_eqm

C⋅(M⋅gc⋅(1 - ec) + mc) + P⋅(gp⋅(1 - ep) + mp) = 0

In [14]:
c_eqm

C⋅(M⋅ec⋅gc - Rc - mc) = 0

## solve each parameter in each equation
### $g_p$: in $dp$ & $dm$

In [19]:
gp_p = solve(p_eqm, gp)
gp_m = solve(m_eqm, gp)
gp_p, gp_m

(Sym[(Rp + mp)/ep], Sym[(-C*M*ec*gc + C*M*gc + C*mc + P*mp)/(P*(ep - 1))])

In [18]:
gp_p

1-element Array{Sym,1}:
 (Rp + mp)/ep

In [8]:
gp_m

1-element Array{Sym,1}:
 (-C*M*ec*gc + C*M*gc + C*mc + P*mp)/(P*(ep - 1))

### $e_p$: in $dp$ & $dm$

In [25]:
ep_p = solve(p_eqm, ep)
ep_m = solve(m_eqm, ep)
ep_p, ep_m

Sym[(Rp + mp)/gp]Sym[(C*(-M*ec*gc + M*gc + mc) + P*gp + P*mp)/(P*gp)]


In [21]:
ep_p

1-element Array{Sym,1}:
 (Rp + mp)/gp

In [22]:
ep_m

1-element Array{Sym,1}:
 (C*(-M*ec*gc + M*gc + mc) + P*gp + P*mp)/(P*gp)

### $R_p$: in $dp$

In [24]:
rp_p = solve(p_eqm, Rp)
rp_p

1-element Array{Sym,1}:
 ep*gp - mp

### $m_p$: in dp & dm

In [26]:
mp_p = solve(p_eqm, mp)
mp_m = solve(m_eqm, mp)
mp_p, mp_m

(Sym[-Rp + ep*gp], Sym[(C*M*ec*gc - C*M*gc - C*mc + P*gp*(ep - 1))/P])

In [27]:
mp_p

1-element Array{Sym,1}:
 -Rp + ep*gp

In [28]:
mp_m

1-element Array{Sym,1}:
 (C*M*ec*gc - C*M*gc - C*mc + P*gp*(ep - 1))/P

### $g_c$: in $dc$ & $dm$

In [29]:
gc_c = solve(c_eqm, gc)
gc_m = solve(m_eqm, gc)
gc_c, gc_m

(Sym[(Rc + mc)/(M*ec)], Sym[(C*mc - P*ep*gp + P*gp + P*mp)/(C*M*(ec - 1))])

In [30]:
gc_c

1-element Array{Sym,1}:
 (Rc + mc)/(M*ec)

In [31]:
gc_m

1-element Array{Sym,1}:
 (C*mc - P*ep*gp + P*gp + P*mp)/(C*M*(ec - 1))

### $e_c$: in $dc$ & $dm$

In [32]:
ec_c = solve(c_eqm, ec)
ec_m = solve(m_eqm, ec)
ec_c, ec_m

(Sym[(Rc + mc)/(M*gc)], Sym[(C*M*gc + C*mc + P*(-ep*gp + gp + mp))/(C*M*gc)])

In [33]:
ec_c

1-element Array{Sym,1}:
 (Rc + mc)/(M*gc)

In [34]:
ec_m

1-element Array{Sym,1}:
 (C*M*gc + C*mc + P*(-ep*gp + gp + mp))/(C*M*gc)

### $R_c$: in $dc$

In [35]:
rc_c = solve(c_eqm, Rc)
rc_c

1-element Array{Sym,1}:
 M*ec*gc - mc

### $m_c$: in $dc$ & $dm$

In [37]:
mc_c = solve(c_eqm, mc)
mc_m = solve(m_eqm, mc)
mc_c, mc_m

(Sym[M*ec*gc - Rc], Sym[(C*M*gc*(ec - 1) + P*ep*gp - P*gp - P*mp)/C])

In [38]:
mc_c

1-element Array{Sym,1}:
 M*ec*gc - Rc

In [39]:
mc_m

1-element Array{Sym,1}:
 (C*M*gc*(ec - 1) + P*ep*gp - P*gp - P*mp)/C